In [ ]:
import numpy as np
import pandas as pd
import random
import torch
import os
seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)

In [ ]:
df = pd.read_csv('data/sampled_msrvtt_clips.csv', header=0)
print(df.shape)        # rows, columns
print(df.columns)      # list of column names
print(df.head())       # preview first few rows

# TODO: Preprocessing the data in CSV format

## Steps needed
- File tải về sẽ là dạng **.mkv**, trong đó có 2 phần:
  - **Audio data**
  - **Frame data**

- Với frame data:
  - Kiểm tra **tính đúng đắn** của dữ liệu → loại bỏ nhiễu/hỏng.
  - Kiểm tra **tính chất** của dữ liệu  
    - Ví dụ: số lượng frame của các video (mean, min, max…).  
    - Có thể vẽ biểu đồ ở bước này.
    - *Một video có thể có nhiều captions → tách các captions này thành các dòng khác nhau trong CSV.*
  - Giảm cấu hình của các frame (ví dụ **720p → 480p**).  
    - Có thể làm ngay lúc download trong file `preprocessing_1` với **yt-dlp** (Em nghien cuu them trong thư vien nay nha).
  - Chuẩn hóa kích thước frame **224 × 224**.  
    - Các bước transform nên để trong `Dataset`.  
    - Việc khai báo `Dataset` thực hiện trong `utils/prepare.py`.
  - Giảm FPS: tạo 2 tùy chọn  
    - 1 bản **không giảm**  
    - 1 bản **giảm** để linh hoạt.

In [ ]:
# TODO: preprocessing the data in csv format
# steps needed
# - file tải về sẽ là dạng .mkv trong đó sẽ có 2 phần
#   + audio data
#   + frame data
# - với frame data
#   + kiem tra tinh dung dan cua du lieu --> n.a nhiễu ....
#   + kiem tra tinh chat cua du lieu  --> ví dụ số lượng frame của các video (mean, min, max...). Em có thể vẽ biểu đồ ở bước này
#    * em có thể thấy 1 video sẽ có nhiều captions. Em có thể tách các captions này thành các dòng khác nhau trong csv.
#   + giam cau hinh cua cac frame (i.e 720p -> 480p). Em có thể làm điều này từ lúc download ở file preprocessing_1 với yt-dlp. Em nghiên cứu thử
#   + chuẩn hóa kích thước frame 224 x 224. Các bước transform em nên để trong Dataset. việc khai báo Dataset thì thực hiện trong utils/prepare.py
#   + với giảm fps, em làm 2 option nha. 1 cái không giảm, 1 cái giảm để mình linh hoạt.#

In [ ]:
import cv2
import numpy as np

def extract_key_frames(video_path, threshold=30):
    """
    Trích xuất key frame dựa trên sự khác biệt giữa các frame liên tiếp.
    Trả về danh sách ảnh grayscale của key frames.
    """
    cap = cv2.VideoCapture(video_path)
    key_frames = []
    ret, prev_frame = cap.read() #read first frame from video, if cannot read video, ret will be False

    if not ret:
        cap.release()
        return key_frames

    prev_gray_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY) #convert key_frame into gray
    key_frames.append(prev_gray_frame)

    while True:
        ret, curr_frame = cap.read() #read others key_frame
        if not ret:
            break
        curr_gray_frame = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        #calculate mean difference between current frame and previous frame
        diff = cv2.absdiff(curr_gray_frame, prev_gray_frame)
        mean_diff = np.mean(diff)
        if mean_diff > threshold:
            key_frames.append(curr_gray_frame)
            prev_gray_frame = curr_gray_frame

    cap.release()
    return key_frames


In [ ]:
def filter_noisy_frames(frames, var_threshold=10):
    """
    Lọc bỏ các frame nhiễu: variance quá thấp.
    """
    filtered = [frame for frame in frames if np.var(frame) > var_threshold]
    return filtered


In [ ]:

keyframes_root = "data/keyframes"
os.makedirs(keyframes_root, exist_ok=True)

In [ ]:
min_check = 50
video_key_frame_counts = []
video_key_frame_paths = []
for idx, row in df.iterrows():
    path = row['clip_path']
    video_id = str(row['video_id'])
    video_dir = os.path.join(keyframes_root, video_id)
    os.makedirs(video_dir, exist_ok=True)
    '''existing_frames = [f for f in os.listdir(video_dir) if f.endswith(".png")]
    if len(existing_frames) >= min_check:
        print(f"Bỏ qua {video_id} (đã có {len(existing_frames)} keyframes).")
        video_key_frame_counts.append(len(existing_frames))
        video_key_frame_paths.append(existing_frames)
        continue'''
    print(f"{idx}" f"Bắt đầu trích xuất key frame của {video_id}")
    frames = extract_key_frames(path)
    frames = filter_noisy_frames(frames)
    frame_paths = []
    for i, frame in enumerate(frames):
        frame_file = os.path.join(video_dir, f"keyframe_{i+1}.png")
        cv2.imwrite(frame_file, frame)
        frame_paths.append(frame_file)
    video_key_frame_counts.append(len(frames))
    video_key_frame_paths.append(frame_paths)
#create new .csv file
df_stats = pd.DataFrame({
    'video_id': df['video_id'],
    'caption' : df['caption'],
    'num_key_frames': video_key_frame_counts,
    'key_frame_paths': video_key_frame_paths
})



In [ ]:
print(df_stats['num_key_frames'].describe())

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.hist(df_stats['num_key_frames'], bins=50, edgecolor='black', linewidth  = 1.2)
plt.xticks(range(0, df_stats['num_key_frames'].max() + 1, 500))
plt.title("Distribution of key frames per video")
plt.xlabel("Number of key frames")
plt.ylabel("Number of videos")
plt.show()


In [ ]:
df = df_stats

# Tính Q1, Q3 và IQR
Q1 = df['num_key_frames'].quantile(0.25)
Q3 = df['num_key_frames'].quantile(0.75)
IQR = Q3 - Q1

# Tính ngưỡng trên dưới
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print(f"Q1 = {Q1}, Q3 = {Q3}, IQR = {IQR}")
print(f"Ngưỡng dưới = {lower_bound}, Ngưỡng trên = {upper_bound}")
if lower_bound <= 0 :
    lower_bound = 1
    print("Chúng ta loại cả những video không có key frame nào. Ngưỡng dưới = 1")
# Lọc dữ liệu
df_filtered = df[(df['num_key_frames'] >= lower_bound) & (df['num_key_frames'] <= upper_bound)]
print(f"Số video ban đầu: {len(df)}, sau khi lọc: {len(df_filtered)}")

print(df_filtered['num_key_frames'].describe())
plt.figure(figsize=(12, 6))
plt.hist(df_filtered['num_key_frames'], bins=50, edgecolor='black', linewidth  = 1.2)
plt.xticks(range(0, df_filtered['num_key_frames'].max() + 1, 500))
plt.yticks(range(0, 51, 5))
plt.title("Distribution of key frames per video")
plt.xlabel("Number of key frames")
plt.ylabel("Number of videos")
plt.show()
df = df_filtered
df.to_csv("data/video_keyggframes.csv")
print(df.head())


In [13]:
import os
import pandas as pd
import ast

# Đường dẫn file input/output
input_csv = "data/sampled_msrvtt_clips.csv"
output_csv = "video_and_keyframe_path.csv"

# Đọc dữ liệu
df = pd.read_csv(input_csv)

data = []

for _, row in df.iterrows():
    video_id = str(row["video_id"])

    captions = row["caption"]

    # Kiểm tra video path
    video_path = f"data/MSR-VTT/{video_id}.mp4"
    if not os.path.exists(video_path):
        continue  # Bỏ qua nếu video không tồn tại

    # Duyệt các keyframes
    keyframe_dir = f"data/keyframes/{video_id}/"
    if not os.path.isdir(keyframe_dir):
        continue  # Bỏ qua nếu folder không có keyframe

    keyframe_paths = sorted([
        os.path.join(keyframe_dir, f)
        for f in os.listdir(keyframe_dir)
        if f.endswith(".png")
    ])

    if len(keyframe_paths) == 0:
        continue  # Bỏ qua nếu không có ảnh nào
    print(video_id)
    # Ghi vào list dữ liệu
    data.append({
        "video_id": video_id,
        "captions": captions,
        "video_path": video_path,
        "keyframe_paths": keyframe_paths,
        "num_key_frames": len(keyframe_paths)
    })

# Ghi CSV mới
out_df = pd.DataFrame(data)
out_df.to_csv(output_csv, index=False)
print(f"✅ Saved {len(out_df)} valid entries to {output_csv}")


video6739
video975
video143
video5587
video2741
video4432
video1270
video1939
video983
video6058
video5712
video1821
video2439
video1528
video4932
video4688
video4755
video960
video6268
video1220
video90
video765
video2212
video4419
video6044
video5130
video4174
video2259
video1072
video2871
video3280
video5761
video5323
video6717
video4767
video6764
video5369
video4561
video1707
video6398
video5200
video2978
video6241
video3582
video6298
video233
video4072
video5255
video963
video3974
video724
video1439
video3680
video6546
video6262
video2898
video4557
video5525
video5013
video6880
video3061
video1414
video2734
video1653
video6700
video819
video1483
video487
video2008
video1168
video6705
video4854
video756
video3453
video3278
video6234
video3250
video5535
video6733
video3022
video2105
video5303
video4985
video3644
video6046
video2465
video192
video4069
video3043
video2304
video1700
video1641
video3089
video3852
video860
video965
video3182
video4567
video416
video722
video5744
video431